In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re

In [25]:
english = []
korean = []
count = 50
with open('korean-english-park.train.en', 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        english.append(line)
        if i-1 == count:
            break

with open('korean-english-park.train.ko', 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        korean.append(line)
        if i-1 == count:
            break

In [27]:
for i in range(len(english)):
    english[i] = re.sub('\n', '', english[i])
for i in range(len(korean)):
    korean[i] = re.sub('\n', '', korean[i])

In [29]:
for i in range(len(english)):
    english[i] = english[i].split()
for i in range(len(korean)):
    korean[i] = korean[i].split()

In [31]:
seq_data = np.stack((english, korean), 1)

In [8]:
word_arr = []
for seq in english:
    word_arr += seq
word_arr += ['<P>']
en_word2num = {c:i for i, c in enumerate(set(word_arr))}
en_num2word = {i:c for i, c in enumerate(en_word2num.keys())}

In [9]:
en_num2word

{0: 'Academy',
 1: 'physical',
 2: 'Southeast',
 3: 'chewing',
 4: 'economy',
 5: '“The',
 6: 'another',
 7: 'sparked',
 8: 'cars.”',
 9: 'has',
 10: 'Korea',
 11: 'indicated',
 12: 'U.N.',
 13: 'accident',
 14: 'commercial',
 15: 'weeks',
 16: 'off',
 17: 'death',
 18: 'storming',
 19: 'Senate',
 20: 'dead',
 21: 'foiled',
 22: 'high',
 23: 'maritime',
 24: 'exploring',
 25: 'nutrition',
 26: 'Stalinist',
 27: 'help',
 28: 'themselves',
 29: 'Swede.',
 30: 'hazardous',
 31: 'longest-running',
 32: 'United',
 33: "It's",
 34: 'suffering',
 35: 'efforts,',
 36: 'than',
 37: 'maintaining',
 38: 'cost',
 39: 'candidate',
 40: 'Rio',
 41: 'postmaster',
 42: 'have',
 43: 'endurance',
 44: 'Chris',
 45: 'economic',
 46: 'surveillance',
 47: 'Researchers',
 48: 'civil',
 49: 'who',
 50: 'pileup',
 51: 'much',
 52: 'in',
 53: 'speed',
 54: 'government',
 55: 'theatre.',
 56: 'injured,',
 57: 'fuel',
 58: 'chartered',
 59: '1.055',
 60: '2,300',
 61: 'alternate',
 62: 'expected',
 63: 'punish',

In [33]:
word_arr = []
for seq in korean:
    word_arr += seq
word_arr += ['<S>', '</S>', '<P>']
ko_word2num = {c:i for i, c in enumerate(set(word_arr))}
ko_num2word = {i:c for i, c in enumerate(ko_word2num.keys())}

In [34]:
ko_word2num

{'"결정적인': 648,
 '"경호': 192,
 '"국방성은': 530,
 '"산업용': 671,
 '"이것보다': 514,
 '"일본,': 357,
 '"자동화': 362,
 '"테러범들과': 127,
 '"특히,': 724,
 '(테러)': 366,
 '(핵)폭탄': 82,
 '100여명의': 416,
 '120대의': 634,
 '150만명': 176,
 '194대의': 255,
 '1970년대와': 389,
 '1994': 156,
 '19년간의': 216,
 '1을': 250,
 '1조550억엔(84억7,000만달러)으로': 324,
 '2,300대의': 711,
 '2000년까지': 692,
 '2000년은': 92,
 '2004년까지': 749,
 '200600밀리그램의,': 33,
 '250억달러에': 512,
 '25명의': 434,
 '2마일': 158,
 '2층의': 333,
 '3개': 722,
 '3분의': 200,
 '40명': 258,
 '4시간': 541,
 '4일간의': 140,
 '5': 77,
 '50대가': 486,
 '50억60억달러로': 370,
 '54명이': 374,
 '5명은': 161,
 '5명이': 630,
 '60대의': 657,
 '65,000명에': 540,
 '6개월간': 558,
 '6층': 548,
 '710': 629,
 '750명': 331,
 '75명의': 508,
 '79.95달러하는': 450,
 '7시': 168,
 '80년대에': 555,
 '9월에': 226,
 ':': 602,
 '</S>': 431,
 '<P>': 448,
 '<S>': 270,
 'Archibald는': 62,
 'Blue': 76,
 'Disco에서': 703,
 'Interstate': 519,
 'Jan': 581,
 'Karlsson이': 460,
 '가까운': 627,
 '가스의': 42,
 '가운데,': 21,
 '가장': 27,
 '가치는': 198,
 '감독할': 251,
 '강한': 420,
 '

In [11]:
def get_max_length(seq_data):
    max_len = 0
    for seq in seq_data:
        if max_len < len(seq):
            max_len = len(seq)
    return max_len

In [12]:
def make_batch(seq_data, enc_max_len, dec_max_len):
    input_batch = []
    output_batch = []
    target_batch = []
    for i, seq in enumerate(seq_data):
        input = []
        output = []
        target = []
        for token in seq[0]:
            input.append(en_word2num[token])
        for _ in range(len(seq[0]), enc_max_len):
            input.append(en_word2num['<P>'])
        input_batch.append(input)
        output.append(ko_word2num['<S>'])
        for token in seq[1]:
            output.append(ko_word2num[token])
            target.append(ko_word2num[token])
        target.append(ko_word2num['</S>'])
        for _ in range(len(seq[1]), dec_max_len):
            output.append(ko_word2num['</S>'])
            target.append(ko_word2num['</S>'])
                
        output_batch.append(output)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [13]:
def get_seq_length(seq_data):
    seq_len = []
    for i, seq in enumerate(seq_data):
        seq_len.append(len(seq))
    return seq_len

In [14]:
learning_rate = 1e-3
n_hidden = 128
max_enc_step = get_max_length(english)
max_dec_step = get_max_length(korean)
n_embedding = 300
total_epoch = 500
batch_size = count
en_dic_len = len(en_word2num)
ko_dic_len = len(ko_word2num)

In [15]:
max_enc_step

43

In [16]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.int32, [None, max_enc_step])
dec_input = tf.placeholder(tf.int32, [None, max_dec_step+1])
targets = tf.placeholder(tf.int64, [None, max_dec_step+1])
W = tf.get_variable(name='encode_embedding', shape=[en_dic_len, n_embedding], trainable=True)
W_ = tf.get_variable(name='decode_embedding', shape=[ko_dic_len, n_embedding], trainable=True)
enc_seq_len = tf.placeholder(dtype=tf.int32, shape=[None])
dec_seq_len = tf.placeholder(dtype=tf.int32, shape=[None])
enc_inputs = tf.nn.embedding_lookup(W, enc_input)
dec_inputs = tf.nn.embedding_lookup(W_, dec_input)

In [17]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    #enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_inputs, sequence_length=enc_seq_len, dtype=tf.float32)

In [18]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    #dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_inputs, initial_state = enc_states, dtype=tf.float32)

In [19]:
logits = tf.layers.dense(outputs, ko_dic_len, activation=None)

In [20]:
outputs.shape

TensorShape([Dimension(None), Dimension(44), Dimension(128)])

In [21]:
cost = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=tf.sequence_mask(dec_seq_len+1, max_dec_step+1, dtype=tf.float32)))
predict = tf.argmax(logits, 2)

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(len(seq_data)/batch_size)
for epoch in range(total_epoch):
    loss_sum = 0
    #print('< epoch:', epoch+1, '>')
    for i in range(total_batch):
        if i == (total_batch-1):
            input_batch, output_batch, target_batch = make_batch(seq_data[i*batch_size:len(seq_data)], max_enc_step, max_dec_step)
            enc_seq_data = get_seq_length(english[i*batch_size:len(seq_data)])
            dec_seq_data = get_seq_length(korean[i*batch_size:len(seq_data)])
        else:
            input_batch, output_batch, target_batch = make_batch(seq_data[i*batch_size:(i+1)*batch_size], max_enc_step, max_dec_step)
            enc_seq_data = get_seq_length(english[i*batch_size:(i+1)*batch_size])
            dec_seq_data = get_seq_length(korean[i*batch_size:(i+1)*batch_size])
        
        _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch, enc_seq_len: enc_seq_data, dec_seq_len: dec_seq_data})
        loss_sum += loss
        #if i % 30 == 29:
        #    print('batch:', '%03d' % (i+1), 'cost =', '{:.6f}'.format(loss_sum/30))
        #    loss_sum = 0
    if epoch % 50 == 49:
        print('epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss_sum/50))
        
print('optimization finished!')

epoch: 050 cost = 0.095094
epoch: 100 cost = 0.037737
epoch: 150 cost = 0.020819
epoch: 200 cost = 0.013141
epoch: 250 cost = 0.007530
epoch: 300 cost = 0.003934
epoch: 350 cost = 0.002143
epoch: 400 cost = 0.001305
epoch: 450 cost = 0.000880
epoch: 500 cost = 0.000638
optimization finished!


In [61]:
def translate(english):
    english = [english]
    # translation(예측)단계에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 <P> 값으로 채운다.(한글의 최대 길이)
    korean = [['<P>']*max_dec_step]
    seq_data = [english + korean]
    # batch 생성
    input_batch, output_batch, target_batch = make_batch(seq_data, max_enc_step, max_dec_step)
    enc_seq_data = get_seq_length(english)
    dec_seq_data = get_seq_length(korean)
    # 학습된 모델로 translation(예측)
    result = sess.run(predict, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch, enc_seq_len: enc_seq_data, dec_seq_len: dec_seq_data})
    # 예측된 단어 인덱스값 가져오기
    decoded = [ko_num2word[j] for j in np.squeeze(result)]
    end = len(decoded)-1
    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    if '</S>' in decoded:
        end = decoded.index('</S>')
    # 출력의 끝을 의미하는 '</S>' 이후의 글자들을 제거하고 문자열로 만든다.
    translated = ' '.join(decoded[:end])
    return translated

In [62]:
for i, seq in enumerate(english):
    temp = ''
    for token in seq:
        temp += token + ' '
    #print(temp)
    print(temp, '\n->', translate(seq), '\n')
    if i == 10:
        break

Much of personal computing is about "can you top this?"  
-> 개인용 컴퓨터 컴퓨터 사용의 사용의 상당 것이며, 것이며, 것이며, 것이며, 동안 동안 동안 동안 예측하고 말하고 말하고 없었다. 없었다. 없었다. 

so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.  
-> 모든 광마우스와 광마우스와 마찬가지 로 책상 책상 책상 책상 위에 위에 위에 마우스 마우스 마우스 외교 외교 동안 예측하고 예측하고 예측하고 예측하고 있다. 

Like all optical mice, But it also doesn't need a desk.  
-> 그러나 이것은 이것은 또한 또한 필요로 필요로 마우스 마우스 마우스 않는다. 예측하고 예측하고 예측하고 있다. 있다. 있다. 있다. 있다. 있다. 

uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.  
-> 79.95달러하는 이 최첨단 무선 광마우스는 허공에서 허공에서 팔목, 팔, 어떤 어떤 움직임에따라 커서의 커서의 움직임을 조절하는 센서를 사용하고 사용하고 사용하고 없었다. 없었다. 있다. 있다. 

Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack.  
-> 정보 관리들은 동남 아시아에서의 선박들에 선박들에 선박들에 (테